### Goal: Mimic the workflow: ETL → analytics → visualisation → optional API.

In [104]:
import pandas as pd
import numpy as np

## Clean Prep Manipulate and get the Dtaset Ready

* Load CSV

* Standardise species names

* Fill missing values (or flag them)

* Convert visit_date to datetime

* Calculate extra useful columns like year, month, age_group

* add other oclumns, datapoints etc, to help the dataset become more realistic and air the analysis

* Save cleaned version back to CSV and/or load into SQLite using df.to_sql().


##### Read and explore

In [105]:
csv_path = "../data/pet_proactive_mock_data.csv"
df = pd.read_csv(csv_path)

print(df.shape)

print("First 10 rows:")
df.head(10)

(700, 8)
First 10 rows:


,pet_id,species,age,weight_kg,condition,treatment,visit_date,cost_gbp
0,P0001,Dog,6.0,10.6,Arthritis,Dental Cleaning,2023-04-08,277.31
1,P0002,Cat,NaN,2.1,NaN,Surgery,2023-11-24,164.88
2,P0003,Dog,NaN,27.6,Gastroenteritis,Diet Consultation,2023-01-01,NaN
3,P0004,Dog,1.0,16.9,Allergy,Vaccination,2023-09-20,NaN
4,P0005,Dog,NaN,26.7,Tumor,Vaccination,2023-12-21,NaN
5,P0006,Bird,16.0,1.1,Allergy,Physiotherapy,2023-10-09,120.23
6,P0007,Dog,14.0,14.1,Dental Disease,Diet Consultation,2023-06-09,419.85
7,P0008,Cat,NaN,2.2,Allergy,Therapy,2023-02-03,196.33
8,P0009,Bird,6.0,NaN,Diabetes,Therapy,2023-06-20,253.33
9,P0010,Rabbit,3.0,4.8,Arthritis,Physiotherapy,2023-09-04,299.70


In [106]:
print("\nData types and missing values:")
print(df.info())



Data types and missing values:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   pet_id      700 non-null    object 
 1   species     700 non-null    object 
 2   age         600 non-null    float64
 3   weight_kg   554 non-null    float64
 4   condition   580 non-null    object 
 5   treatment   623 non-null    object 
 6   visit_date  700 non-null    object 
 7   cost_gbp    661 non-null    float64
dtypes: float64(3), object(5)
memory usage: 43.9+ KB
None


In [107]:
print("\nBasic stats:")
print(df.describe())


Basic stats:
              age   weight_kg    cost_gbp
count  600.000000  554.000000  661.000000
mean     9.486667    8.757220  218.244766
std      5.931424    7.927665  130.832967
min      0.000000    0.500000    0.140000
25%      4.000000    1.700000  113.940000
50%     10.000000    6.250000  213.440000
75%     15.000000   14.300000  307.640000
max     19.000000   34.900000  589.660000


In [108]:
#check unique values for categorical columns
print("\nUnique species:", df['species'].unique())
print("Unique conditions:", df['condition'].unique())
print("Unique treatments:", df['treatment'].unique())


Unique species: ['Dog' 'Cat' 'Bird' 'Rabbit']
Unique conditions: ['Arthritis' nan 'Gastroenteritis' 'Allergy' 'Tumor' 'Dental Disease'
 'Diabetes' 'Obesity' 'Skin Infection' 'Ear Infection' 'Hip Dysplasia']
Unique treatments: ['Dental Cleaning' 'Surgery' 'Diet Consultation' 'Vaccination'
 'Physiotherapy' 'Therapy' nan 'Check-up' 'Medication']


In [109]:
print("Unique Id-s:", df['pet_id'].unique().shape)

Unique Id-s: (700,)


#### start cleaning and prepping

In [110]:
# Standardise categorical data
# species column: make everything lowercase then capitalise:

df['species'] = df['species'].str.strip().str.lower().str.capitalize()
df['condition'] = df['condition'].str.strip().str.lower().str.capitalize()
df['treatment'] = df['treatment'].str.strip().str.lower().str.capitalize()


In [111]:
# Handle missing numeric data
# For age and weight_kg:
# Decide whether to fill with median, mean, or flag as missing.

df['age'] = df['age'].fillna(df['age'].mean())
df['weight_kg'] = df['weight_kg'].fillna(df['weight_kg'].mean())


# Optional: add a boolean column like age_missing if you want to track originally missing values.

In [112]:
# Handle missing categorical data
df['condition'] = df['condition'].fillna("Unknown")
df['treatment'] = df['treatment'].fillna("Unknown")

In [113]:
# fix numerical values - cannot be negative
# dfl.oc[...,...] = np.nan fixes the new values in place, doesn't rewrite the df
df.loc[df['weight_kg'] <= 0, 'weight_kg'] = np.nan
df['weight_kg'] = df['weight_kg'].fillna(df['weight_kg'].mean())


In [114]:
# cost cannot be negative, fill missing values with na
df.loc[df['cost_gbp'] < 0, 'cost_gbp'] = np.nan
df['cost_gbp'] = df['cost_gbp'].fillna(df['cost_gbp'].mean())

In [115]:
df['visit_date'] = pd.to_datetime(df['visit_date'], errors='coerce')

In [147]:
# Ensure 'visit_date' is in datetime format
df['visit_date'] = pd.to_datetime(df['visit_date'], errors='coerce')

# Extract unique years from the 'visit_date' column
unique_years = df['visit_date'].dt.year.unique()

# Convert to a list (optional)
unique_years_list = unique_years.tolist()

print(unique_years_list)


[2023]


In [116]:
# extracting more cols for analysis
df['year'] = df['visit_date'].dt.year
df['month'] = df['visit_date'].dt.month

In [117]:
# pd.cut segments continuous numeric data into discrete “bins” or intervals.
# for easier visuals

# bins are the edges of the intervals. 

bins = [0, 2, 5, 10, 15, 20, np.inf]  # np.inf = all ages above 20
labels = ['0-2','3-5','6-10','11-15','16-20','20+']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels)

In [118]:
# grup by the avg cost per condition:
# Make sure cost_gbp is numeric
df['cost_gbp'] = pd.to_numeric(df['cost_gbp'], errors='coerce')

# Calculate average cost per condition
avg_cost_per_condition = df.groupby('condition')['cost_gbp'].mean().sort_values(ascending=False)

In [119]:
df.head()

,pet_id,species,age,weight_kg,condition,treatment,visit_date,cost_gbp,year,month,age_group
0,P0001,Dog,6.000000,10.6,Arthritis,Dental cleaning,2023-04-08,277.310000,2023,4,6-10
1,P0002,Cat,9.486667,2.1,Unknown,Surgery,2023-11-24,164.880000,2023,11,6-10
2,P0003,Dog,9.486667,27.6,Gastroenteritis,Diet consultation,2023-01-01,218.244766,2023,1,6-10
3,P0004,Dog,1.000000,16.9,Allergy,Vaccination,2023-09-20,218.244766,2023,9,0-2
4,P0005,Dog,9.486667,26.7,Tumor,Vaccination,2023-12-21,218.244766,2023,12,6-10


#### manipulate the dataset a bit, since it is mock dataset, I can add more data or twist it in a way to try and mimic real world data at least remotely.

Just noticed my mockdb does not have any pets with repeated visits, so will change it now-

In [123]:
dogs = df[df['species'] == 'Dog']
cats = df[df['species'] == 'Cat']

# randomly choose pets
np.random.seed(44)
dog_ids_to_repeat = np.random.choice(dogs['pet_id'], size=27, replace=False)
cat_ids_to_repeat = np.random.choice(cats['pet_id'], size=32, replace=False)


def duplicate_visits(df, pet_ids, num_duplicates=1):
    new_rows = []
    for pet_id in pet_ids:
        original_row = df[df['pet_id'] == pet_id].iloc[0]
        for _ in range(num_duplicates):
            new_row = original_row.copy()
            # Randomize visit_date within 2023 i.e.
            new_row['visit_date'] = pd.to_datetime('2023-01-01') + pd.to_timedelta(
                np.random.randint(0, 365), unit='days'
            )
            new_row['cost_gbp'] = max(5, new_row['cost_gbp'] * np.random.uniform(0.8, 1.2))
            new_rows.append(new_row)
    return pd.DataFrame(new_rows)

# call the function on dogs and cats lists
dog_duplicates = duplicate_visits(dogs, dog_ids_to_repeat, num_duplicates=7) 
cat_duplicates = duplicate_visits(cats, cat_ids_to_repeat, num_duplicates=4)

# Combine with original df
df_new = pd.concat([df, dog_duplicates, cat_duplicates], ignore_index=True)

# Sort by pet_id for clarity
df_new = df_new.sort_values(by='pet_id').reset_index(drop=True)


In [124]:
df_new.shape

(1017, 11)

In [125]:
print("Unique Id-s:", df_new['pet_id'].unique().shape)
print("Overall Id-s:", df_new['pet_id'].shape)

Unique Id-s: (700,)
Overall Id-s: (1017,)


Adding a new column called insurance to help the analysis:

In [131]:
def assign_insurance(row):
    species = row['species'].lower()
    age = row['age']

    if species == "dog":
        base_prob = 0.65
    elif species == "cat":
        base_prob = 0.35
    else:  # exotic, rabbit, etc
        base_prob = 0.10

    # Adjust for age (younger pets more likely insured, older less)
    if age <= 3:
        prob = base_prob + 0.10  # bump up for young
    elif age >= 10:
        prob = base_prob - 0.15  # drop for old
    else:
        prob = base_prob

    # Keep probs within [0.05, 0.9]
    prob = min(max(prob, 0.05), 0.9)

    return np.random.rand() < prob

# Add insurance column
df_new['insured'] = df_new.apply(assign_insurance, axis=1)


In [132]:
df_new['insured']

0        True
1       False
2        True
3        True
4        True
        ...  
1012    False
1013    False
1014    False
1015    False
1016    False
Name: insured, Length: 1017, dtype: bool

In [133]:
plans = {
    "Accident-only": (0.5, 0.7),
    "Time-limited": (0.6, 0.8),
    "Maximum benefit": (0.7, 0.85),
    "Lifetime": (0.8, 0.9),
}

# assign plans to insured pets
df_new["insurance_plan"] = np.where(
    df_new["insured"],
    np.random.choice(list(plans.keys()), size=len(df_new), p=[0.2, 0.3, 0.3, 0.2]),
    "None"
)

# coverage rate based on plan
df_new["coverage_rate"] = df_new.apply(
    lambda row: np.random.uniform(*plans[row["insurance_plan"]])
    if row["insurance_plan"] != "None" else 0,
    axis=1
)

# calculate insurance economics
df_new["covered_gbp"] = (df_new["cost_gbp"] * df_new["coverage_rate"]).round(2)
df_new["out_of_pocket_gbp"] = (df_new["cost_gbp"] - df_new["covered_gbp"]).round(2)


save the new dataframe with added data to a new clean CSV

In [134]:
# Save updated CSV
df_new.to_csv("../data/cleaned_pet_proactive_mock_data.csv", index=False)

print("Updated CSV with repeated visits created!")
csv_path = "../data/cleaned_pet_proactive_mock_data.csv" 
df = pd.read_csv(csv_path)

Updated CSV with repeated visits created!


In [135]:
df.shape

(1017, 16)

In [137]:
df.iloc[:, 11:]

,insured,insurance_plan,coverage_rate,covered_gbp,out_of_pocket_gbp
0,True,Lifetime,0.830259,230.24,47.07
1,False,NaN,0.000000,0.00,164.88
2,True,Lifetime,0.893684,195.04,23.20
3,True,Accident-only,0.532569,116.23,102.01
4,True,Accident-only,0.637794,143.32,81.39
...,...,...,...,...,...
1012,False,NaN,0.000000,0.00,5.00
1013,False,NaN,0.000000,0.00,1.17
1014,False,NaN,0.000000,0.00,5.00
1015,False,NaN,0.000000,0.00,5.00


In [138]:
df[['coverage_rate', 'covered_gbp', 'out_of_pocket_gbp', 'cost_gbp']]

,coverage_rate,covered_gbp,out_of_pocket_gbp,cost_gbp
0,0.830259,230.24,47.07,277.310000
1,0.000000,0.00,164.88,164.880000
2,0.893684,195.04,23.20,218.244766
3,0.532569,116.23,102.01,218.244766
4,0.637794,143.32,81.39,224.709281
...,...,...,...,...
1012,0.000000,0.00,5.00,5.000000
1013,0.000000,0.00,1.17,1.170000
1014,0.000000,0.00,5.00,5.000000
1015,0.000000,0.00,5.00,5.000000


### Generate table stats for to get the first insights about data. before conducting full analysis:

In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017 entries, 0 to 1016
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pet_id             1017 non-null   object 
 1   species            1017 non-null   object 
 2   age                1017 non-null   float64
 3   weight_kg          1017 non-null   float64
 4   condition          1017 non-null   object 
 5   treatment          1017 non-null   object 
 6   visit_date         1017 non-null   object 
 7   cost_gbp           1017 non-null   float64
 8   year               1017 non-null   int64  
 9   month              1017 non-null   int64  
 10  age_group          984 non-null    object 
 11  insured            1017 non-null   bool   
 12  insurance_plan     330 non-null    object 
 13  coverage_rate      1017 non-null   float64
 14  covered_gbp        1017 non-null   float64
 15  out_of_pocket_gbp  1017 non-null   float64
dtypes: bool(1), float64(6), 

In [140]:
df.describe()

,age,weight_kg,cost_gbp,year,month,coverage_rate,covered_gbp,out_of_pocket_gbp
count,1017.000000,1017.000000,1017.000000,1017.0,1017.000000,1017.000000,1017.000000,1017.000000
mean,9.307847,9.889311,208.179833,2023.0,6.706981,0.238405,48.317925,159.861681
std,5.342660,7.275665,129.384393,0.0,3.385745,0.348567,87.389286,132.926079
min,0.000000,0.500000,0.140000,2023.0,1.000000,0.000000,0.000000,0.140000
25%,5.000000,4.300000,106.170000,2023.0,4.000000,0.000000,0.000000,45.700000
50%,9.486667,8.757220,209.668084,2023.0,7.000000,0.000000,0.000000,124.170000
75%,13.000000,13.800000,293.178439,2023.0,10.000000,0.646257,64.760000,250.420000
max,19.000000,34.900000,672.906176,2023.0,12.000000,0.899248,477.250000,672.910000


In [141]:

# Count of categories (for categorical columns)
print("\n=== Category Counts ===")
for col in df.select_dtypes(include="object").columns:
    print(f"\n{col} value counts:")
    print(df[col].value_counts())


=== Category Counts ===

pet_id value counts:
pet_id
P0577    8
P0484    8
P0463    8
P0116    8
P0668    8
        ..
P0248    1
P0249    1
P0250    1
P0251    1
P0351    1
Name: count, Length: 700, dtype: int64

species value counts:
species
Dog       365
Cat       307
Bird      174
Rabbit    171
Name: count, dtype: int64

condition value counts:
condition
Unknown            180
Ear infection      117
Gastroenteritis     93
Dental disease      91
Arthritis           87
Obesity             81
Diabetes            78
Skin infection      75
Allergy             74
Tumor               71
Hip dysplasia       70
Name: count, dtype: int64

treatment value counts:
treatment
Dental cleaning      169
Physiotherapy        154
Vaccination          132
Diet consultation    110
Surgery              101
Unknown               99
Therapy               93
Check-up              91
Medication            68
Name: count, dtype: int64

visit_date value counts:
visit_date
2023-09-16    10
2023-04-14     8
20

In [142]:
# Missing values check
print("\n=== Missing Values ===")
print(df.isnull().sum())


=== Missing Values ===
pet_id                 0
species                0
age                    0
weight_kg              0
condition              0
treatment              0
visit_date             0
cost_gbp               0
year                   0
month                  0
age_group             33
insured                0
insurance_plan       687
coverage_rate          0
covered_gbp            0
out_of_pocket_gbp      0
dtype: int64


In [146]:
print(df.columns.tolist())

['pet_id', 'species', 'age', 'weight_kg', 'condition', 'treatment', 'visit_date', 'cost_gbp', 'year', 'month', 'age_group', 'insured', 'insurance_plan', 'coverage_rate', 'covered_gbp', 'out_of_pocket_gbp']


In [144]:
# groupby summaries
if "species" in df.columns and "cost_gbp" in df.columns:
    print("\n=== Avg cost per species ===")
    print(df.groupby("species")["cost_gbp"].agg(["mean", "median", "min", "max", "count"]))

if "treatment" in df.columns and "cost_gbp" in df.columns:
    print("\n=== Avg cost per treatment ===")
    print(df.groupby("treatment")["cost_gbp"].agg(["mean", "median", "min", "max", "count"]))



=== Avg cost per species ===
               mean      median   min         max  count
species                                                 
Bird     220.186708  218.244766  7.55  589.660000    174
Cat      198.376905  195.495986  0.32  672.906176    307
Dog      210.007012  210.135931  0.14  584.610000    365
Rabbit   209.661599  211.320000  2.10  580.800000    171

=== Avg cost per treatment ===
                         mean      median    min         max  count
treatment                                                          
Check-up           262.778829  267.300000   2.41  589.660000     91
Dental cleaning    177.042652  166.620000   1.09  541.000000    169
Diet consultation  204.467497  176.270000   0.49  672.906176    110
Medication         238.658578  218.244766  25.18  507.075148     68
Physiotherapy      192.509894  210.520143   7.08  442.820000    154
Surgery            185.690681  179.070000   0.14  580.800000    101
Therapy            196.386843  216.680000   0.32  53

## step 2

Data analysis & visualisation

Focus on insights they’d care about:

Most common conditions by species

Average treatment costs by condition and age group

Cost trends over time

Distribution of weights/ages

Use matplotlib/seaborn.

Optional: build small dashboards in Streamlit or Flask so someone can filter by species, condition, or age and see charts update dynamically.

This shows you know their analytics workflow: Python data model + visualisation.

## Step 3
Optional AI magic (to impress)

Quick win: LLM-powered query helper

Example: Ask “What is the average diabetes cost for cats under 5?” → translate to pandas query → show output.

Lightweight, no need for heavy training, and screams “tech curiosity” without overcomplicating.

Computer vision is optional but would be flashy; if you’re short on time, skip it for now.

## Step 4
API exposure (mini production flair)

Wrap a simple Flask API around your cleaned data:

Endpoint /pet_summary?species=Dog&condition=Arthritis → returns JSON with summary stats.

Optional for portfolio, but if done, it demonstrates familiarity with their API workflow.

## Step 5

Documentation / presentation

GitHub repo + README:

Explain the dataset and ETL steps

Show visualisations as screenshots

Optional: screenshots or a deployed Streamlit/Flask app link

Title: Pet Health Analytics Demo

Include 1–2 sentences on why this mirrors Pet Proactive’s real workflow.